In [1]:
import pandas as pd 
import os
import numpy as np
from tqdm import tqdm
import subprocess
import shutil
import zipfile
from multiprocessing import Pool


In [ ]:
from multiprocessing.dummy import Pool as ThreadPool
import os
print("PATH:", os.environ['PATH'])
def convert_mp4_to_zip_ffmpeg(mp4_path:str, 
                              zip_path:str,
                              fps:int=None,
                              uniform_num_frames:int=None,
                              scale: int = None,
                              start_time: int = 0,
                              quality: int = 2,
                              duration: int = None,
                              override: bool = True):
    # mp4_path: Đường dẫn đến file video MP4 cần xử lý
    # zip_path: Đường dẫn đến file ZIP đầu ra, nơi sẽ chứa các hình ảnh được trích xuất
    # fps: Số khung hình trên giây để trích xuất (mặc định: fps gốc của video)
    # uniform_num_frames: Số lượng khung hình đồng đều cần trích xuất (ghi đè lên fps nếu được chỉ định)
    # scale: Kích thước mới cho hình ảnh đầu ra (nếu được chỉ định)
    # start_time: Thời điểm bắt đầu trong video để trích xuất hình ảnh (tính bằng giây)
    # quality: Chất lượng của hình ảnh JPEG đầu ra (2 là chất lượng tốt)
    # duration: Khoảng thời gian cần trích xuất từ video (tính bằng giây, mặc định: toàn bộ video)
    # override: Nếu True, ghi đè lên file ZIP đã tồn tại; nếu False, bỏ qua xử lý nếu file ZIP đã tồn tại

    # Kiểm tra xem file ZIP đã tồn tại hay không
    if os.path.isfile(zip_path):
        if override:
            os.remove(zip_path)
        else:
            return 0
        
    #Lấy thư mục chứa file ZIP:
    output_dir = os.path.dirname(zip_path)
    tmp_dir = os.path.join(output_dir, '{}_tmp'.format(os.path.basename(zip_path)[:-4]))
    if not os.path.isdir(tmp_dir):
        os.makedirs(tmp_dir, exist_ok=True)
    
    cmd = f"ffmpeg -nostdin -y -i {mp4_path} -start_number 0 -q 0 -vf fps=1 {tmp_dir}/%07d.jpg -threads {os.cpu_count()}"
    
    print(f"Đang xử lý: {mp4_path}")
    print(f"Lệnh ffmpeg: {cmd}")
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        print(f"Kết quả ffmpeg: {result.stderr}")
        if result.returncode != 0:
            print(f"Lỗi ffmpeg: {result.stderr}")
            return -1
    except Exception as e:
        print(f"Lỗi khi chạy ffmpeg: {str(e)}")
        return -1
    
    try:
        with open(os.devnull, "w") as null:
            subprocess.call(cmd, shell=True, timeout=60, stderr=null)
    except subprocess.TimeoutExpired:
        shutil.rmtree(tmp_dir)
        return -1
    else:
        imgpaths = [os.path.join(tmp_dir, fn) for fn in os.listdir(tmp_dir) if fn.endswith('.jpg')]
        #inds = np.linspace(0, len(imgpaths) - 1, uniform_num_frames, dtype=int).tolist()
        #imgpaths = [imgpaths[i] for i in inds]
        if len(imgpaths) != 0:
            with zipfile.ZipFile(zip_path, 'w') as wzip:
                for p in imgpaths:
                    name = p.split("/")[-1]
                    wzip.write(p, arcname=name)
        shutil.rmtree(tmp_dir)

    return 0

def do_convert(args):
    # mode, token, idx, fps = args
    mode, token, idx, fps = args
    video_path = "../data/videos/%s/%s/%s.mp4" % (mode, token, idx)
    zip_path = "../data/jpg_zips/%s/%s.zip" % (idx[-2:], idx)
    convert_mp4_to_zip_ffmpeg(video_path, zip_path, fps=fps)

def main():
    train_query_meta = "..\data\meta\\train\\train_query_metadata.csv"
    train_ref_meta = "..\data\meta\\train\\train_reference_metadata.csv"
    test_query_meta = "..\data\meta\\test\\test_query_metadata.csv"
    test_ref_meta = "..\data\meta\\test\\test_reference_metadata.csv"

    meta_query = pd.read_csv(train_query_meta)
    meta_ref = pd.read_csv(train_ref_meta)
    meta_query_ = pd.read_csv(test_query_meta, encoding = 'unicode_escape')
    meta_ref_ = pd.read_csv(test_ref_meta, encoding = 'unicode_escape')

    meta = pd.concat([meta_query, meta_ref], ignore_index=True)

    meta = pd.concat([meta_query, meta_ref], ignore_index=True)
    meta["mode"] = "train"

    meta_ = pd.concat([meta_query_, meta_ref_], ignore_index=True)
    meta_["mode"] = "test"


    meta = pd.concat([meta, meta_], ignore_index=True)
    meta = meta.drop(columns = ['users', 'urls'])

    all_videos_id = # File txt tất cả vid id

    with open(f"..\data\meta\\test\\{all_videos_id}", "r") as f:
        available_videos = [line.strip() for line in f.readlines()]
    meta1 = meta[meta["video_id"].isin(available_videos)]

    video_list = meta1["video_id"].tolist()
    modes = meta1['mode'].tolist()

    args_list = []
    for idx,m in zip(video_list, modes):
        t = "query" if idx.startswith('Q') else "reference"
        args_list.append((m, t, idx, 1))



    print("#####", len(video_list))

    with ThreadPool(os.cpu_count()) as pool:  # Sử dụng số lượng luồng bằng số lõi CPU
        pool.map(do_convert, tqdm(args_list))
    pool.join()

if __name__ == "__main__":
    main()    
    
#     # Tạ thư mục đầu ra nếu không tồn tại


PATH: e:\FeipengMa6 VSC22-Submission main VSC22-Descriptor-Track-1st\env39\Scripts;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\ProgramData\chocolatey\bin;C:\Program Files\Git\cmd;C:\Program Files\Cloudflare\Cloudflare WARP\;C:\Program Files\Go\bin;C:\Program Files\nodejs\;C:\Users\huyvu\AppData\Local\Programs\Python\Python39\Scripts\;C:\Users\huyvu\AppData\Local\Programs\Python\Python39\;C:\Users\huyvu\AppData\Local\Programs\Python\Python312\Scripts\;C:\Users\huyvu\AppData\Local\Programs\Python\Python312\;C:\Program Files\MySQL\MySQL Shell 8.0\bin\;C:\Users\huyvu\anaconda3;C:\Users\huyvu\anaconda3\Library\mingw-w64\bin;C:\Users\huyvu\anaconda3\Library\usr\bin;C:\Users\huyvu\anaconda3\Library\bin;C:\Users\huyvu\anaconda3\Scripts;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32

100%|██████████| 224/224 [00:00<00:00, 112115.05it/s]

Đang xử lý: ../data/videos/test/query/Q301064.mp4
Lệnh ffmpeg: ffmpeg -nostdin -y -i ../data/videos/test/query/Q301064.mp4 -start_number 0 -q 0 -vf fps=1 ../data/jpg_zips/64\Q301064_tmp/%07d.jpg -threads 16
Đang xử lý: ../data/videos/test/query/Q304367.mp4
Lệnh ffmpeg: ffmpeg -nostdin -y -i ../data/videos/test/query/Q304367.mp4 -start_number 0 -q 0 -vf fps=1 ../data/jpg_zips/67\Q304367_tmp/%07d.jpg -threads 16
Đang xử lý: ../data/videos/test/query/Q306255.mp4
Lệnh ffmpeg: ffmpeg -nostdin -y -i ../data/videos/test/query/Q306255.mp4 -start_number 0 -q 0 -vf fps=1 ../data/jpg_zips/55\Q306255_tmp/%07d.jpg -threads 16
Đang xử lý: ../data/videos/test/query/Q302932.mp4
Lệnh ffmpeg: ffmpeg -nostdin -y -i ../data/videos/test/query/Q302932.mp4 -start_number 0 -q 0 -vf fps=1 ../data/jpg_zips/32\Q302932_tmp/%07d.jpg -threads 16
Đang xử lý: ../data/videos/test/query/Q301749.mp4
Lệnh ffmpeg: ffmpeg -nostdin -y -i ../data/videos/test/query/Q301749.mp4 -start_number 0 -q 0 -vf fps=1 ../data/jpg_zips/4

Kết quả ffmpeg: ffmpeg version 6.0-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore